In [1]:
!pip install ultralytics


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import deque

# -----------------------------
# UPDATE YOUR VIDEO PATH
# -----------------------------
VIDEO_PATH = r"C:\Users\aparn\Downloads\CV\volleyball_match.mp4"
OUTPUT_PATH = r"C:\Users\aparn\Downloads\CV\final_output.avi"

# Load YOLO model
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(VIDEO_PATH)

if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Windows-friendly codec
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(OUTPUT_PATH, fourcc, fps, (width, height))

trail = deque(maxlen=30)

frame_center_x = width // 2

while True:
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False)[0]

    ball_center = None
    left_count = 0
    right_count = 0

    for box in results.boxes:
        cls_id = int(box.cls[0])
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # COCO class 32 = sports ball
        if cls_id == 32:
            ball_center = (int((x1 + x2) / 2),
                           int((y1 + y2) / 2))
            cv2.circle(frame, ball_center, 10,
                       (0, 255, 255), 3)

        # COCO class 0 = person
        if cls_id == 0:
            person_center_x = int((x1 + x2) / 2)

            if person_center_x < frame_center_x:
                left_count += 1
                color = (255, 0, 0)
            else:
                right_count += 1
                color = (0, 255, 0)

            cv2.rectangle(frame,
                          (x1, y1),
                          (x2, y2),
                          color, 2)

    # Track ball trail
    trail.appendleft(ball_center)

    for i in range(1, len(trail)):
        if trail[i - 1] is None or trail[i] is None:
            continue
        thickness = int(np.sqrt(30 / float(i + 1)) * 2)
        cv2.line(frame,
                 trail[i - 1],
                 trail[i],
                 (0, 0, 255),
                 thickness)

    # Draw divider
    cv2.line(frame,
             (frame_center_x, 0),
             (frame_center_x, height),
             (255, 255, 255), 2)

    cv2.putText(frame,
                f"Left: {left_count}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 0, 0),
                2)

    cv2.putText(frame,
                f"Right: {right_count}",
                (width - 220, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2)

    cv2.imshow("YOLO Volleyball Tracking", frame)

    out.write(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("Processing complete.")
print("Saved as:", OUTPUT_PATH)
